<a href="https://colab.research.google.com/github/ASHIKalip/object_detection/blob/main/majpre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%config IPCompleter.greedy=True

In [10]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/dropcreations/FFmpeg-for-Google-Colab/1bd5d8aeb88c8f402a9aea4846fab6304e7d8e43/Google-Drive-Logo.svg' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3><b>Mount Google Drive</b></h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


In [13]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/dropcreations/FFmpeg-for-Google-Colab/32abf44ff4c8d145a94a24611f01141926a8daaa/FFmpeg-Logo.svg' height="40" alt="FFmpeg-logo"/></center>
#@markdown <center><h3><b>Install FFmpeg</b></h3></center><br>
from IPython.display import clear_output
!sudo curl -L https://github.com/BtbN/FFmpeg-Builds/releases/download/latest/ffmpeg-master-latest-linux64-gpl.tar.xz -o /usr/local/bin/ffmpeg.tar.xz
clear_output()
%cd /usr/local/bin/
clear_output()
!7z e /usr/local/bin/ffmpeg.tar.xz
clear_output()
!7z e /usr/local/bin/ffmpeg.tar
clear_output()
!sudo chmod a+rx /usr/local/bin/ffmpeg
clear_output()
%cd /content/
!sudo curl -L https://mkvtoolnix.download/appimage/MKVToolNix_GUI-70.0.0-x86_64.AppImage -o /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage
!sudo chmod u+rx /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage
!sudo ln -s /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage /usr/local/bin/mkvmerge
!sudo chmod a+rx /usr/local/bin/mkvmerge
clear_output()
print("Successfully Installed.")

/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0^C
ln: failed to create symbolic link '/usr/local/bin/mkvmerge': File exists
Successfully Installed.


In [15]:
!pip install ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=086ad689c5ec569664bb5d573ef5f52a2a25447470ca36b891a8bc6bb074cb8a
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg


In [16]:
import librosa
import librosa.display
import numpy as np
import pandas as pd
import ffmpeg as ff
import os
from typing import List, Tuple, Dict
import matplotlib.pyplot as plt
import random
import cv2
import pickle
import datetime

In [33]:
def extract_audio_from_video(file_path: str) -> np.ndarray:
    inputfile = ff.input(file_path)
    out = inputfile.output('-', format='f32le', acodec='pcm_f32le', ac=1, ar='44100')
    raw = out.run(capture_stdout=True)
    del inputfile, out
    return np.frombuffer(raw[0],np.float32)
# import os
# import json
# import subprocess
# import prettytable

# inputFile = "/content/drive/MyDrive/Big5/train/0ObEtA6Q8vU.005.mp4" #@param {type:"string"}
# saveToSourceLocation = True #@param {type:"boolean"}

# jsonFFprobe = subprocess.check_output([
#     'ffprobe',
#     '-hide_banner',
#     '-loglevel',
#     'warning',
#     '-print_format',
#     'json',
#     '-show_entries',
#     'stream',
#     '-i',
#     os.path.abspath(inputFile),
#     ], stderr=subprocess.DEVNULL)
# jsonData = json.loads(jsonFFprobe)

# streamCount = len(jsonData['streams'])
# commandLine = '-hide_banner -i "' + inputFile + '" '

# codecTable = prettytable.PrettyTable()
# codecTable.field_names = ['Track ID', 'Codec name', 'Channels', 'Sample rate']
# codecTable.align = 'l'

# for i in range(streamCount):
#     codec_type = jsonData.get('streams')[int(i)].get('codec_type')
#     if codec_type == "audio":
#         index = jsonData.get('streams')[int(i)].get('index')
#         codec_name = jsonData.get('streams')[int(i)].get('codec_name')
#         channel_layout = jsonData.get('streams')[int(i)].get('channels')
#         sample_rate = jsonData.get('streams')[int(i)].get('sample_rate')
#         codecTable.add_row([index, codec_name, channel_layout, sample_rate])

# stringTable = '\t' + codecTable.__str__().replace('\n', '\n\t')
# print(stringTable)

# trackID = input("\ntrackID: ")
# codec_name = jsonData.get('streams')[int(trackID)].get('codec_name')
# if "opus" in codec_name:
#     codec_ext = "ogg"
# elif "dts" in codec_name:
#     codec_ext = "mka"
# elif "aac" in codec_name:
#     codec_ext = "aac"
# elif "truehd" in codec_name:
#     codec_ext = "mka"
# elif codec_name == "ac3":
#     codec_ext = "ac3"
# elif codec_name == "eac3":
#     codec_ext = "eac3"
# elif "pcm" in codec_name:
#     codec_ext = "wav"
# elif "flac" in codec_name:
#     codec_ext = "flac"

# commandLine = commandLine + "-vn -sn -map 0:" + str(trackID) + " -c copy "

# sourceName = os.path.splitext(os.path.basename(os.path.abspath(inputFile)))[0]
# sourceFolder = os.path.dirname(os.path.abspath(inputFile))

# if saveToSourceLocation is True:
#     outputFile = os.path.join(sourceFolder, (sourceName + f'_[{codec_name}].' + codec_ext))
#     commandLine = commandLine + '"' + outputFile + '"'
# else:
#     outputFolder = input("outputFolder: ")
#     outputFile = os.path.join(outputFolder, (sourceName + f'_[{codec_name}].' + codec_ext))
#     commandLine = commandLine + '"' + outputFile + '"'

# !ffmpeg {commandLine}

In [18]:
def preprocess_audio_series(raw_data: np.ndarray) -> np.ndarray:
    N, M = 24, 1319
    mfcc_data = librosa.feature.mfcc(raw_data, n_mfcc= 24)
    
    #Getting spectral mean (centroid)
    #mean = librosa.feature.spectral_centroid(result)
    
    #Standardizing MFCC (zero mean and unit variance)
    mfcc_data_standardized = (mfcc_data - np.mean(mfcc_data)) / np.std(mfcc_data)
    
    # Use pre-padding (Note: with 0, which is also the mean after standardization) to unify the length of the samples.
    number_of_columns_to_fill = M - mfcc_data_standardized.shape[1]
    padding = np.zeros((N,number_of_columns_to_fill))
    
    padded_data = np.hstack((padding, mfcc_data_standardized))
    
    #Reshaping to N,M,1
    return padded_data.reshape(N,M,1)
    

In [19]:
def get_number_of_frames(file_path: str) -> int:
    probe = ff.probe(filePath)
    video_streams = [stream for stream in probe["streams"] if stream["codec_type"] == "video"]
    #width = video_streams[0]['coded_width']
    #height = video_streams[0]['coded_height']
    del probe
    return video_streams[0]['nb_frames']

In [20]:
def extract_N_video_frames(file_path: str, number_of_samples: int = 6) -> List[np.ndarray]:
    nb_frames = int(get_number_of_frames(file_path= filePath))
    
    video_frames = []
    random_indexes = random.sample(range(0, nb_frames), number_of_samples)
    
    cap = cv2.VideoCapture(filePath)
    for ind in random_indexes:
        cap.set(1,ind)
        res, frame = cap.read()
        video_frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    cap.release()
    del cap, random_indexes
    return video_frames

In [21]:
def resize_image(image: np.ndarray, new_size: Tuple[int,int]) -> np.ndarray:
    return cv2.resize(image, new_size, interpolation = cv2.INTER_AREA)

In [22]:
def crop_image_window(image: np.ndarray, training: bool = True) -> np.ndarray:
    height, width, _ = image.shape
    if training:
        MAX_N = height - 128
        MAX_M = width - 128
        rand_N_index, rand_M_index = random.randint(0, MAX_N) , random.randint(0, MAX_M)
        return image[rand_N_index:(rand_N_index+128),rand_M_index:(rand_M_index+128),:]
    else:
        N_index = (height - 128) // 2
        M_index = (width - 128) // 2
        return image[N_index:(N_index+128),M_index:(M_index+128),:]

In [23]:
def reading_label_data(file_name: str, dictionary: Dict[str,str]) -> np.ndarray:
    features = ['extraversion', 'neuroticism', 'agreeableness', 'conscientiousness', 'openness']
    extracted_data = [float(dictionary[label][file_name]) for label in features]
    return np.stack(extracted_data).reshape(5,1)

In [50]:
def preprocessing_input(file_path: str, file_name: str, dictionary: Dict[str,str], training: bool = True) -> Tuple[np.ndarray,np.ndarray,np.ndarray]:
    #Audio
    #extracted_audio_raw = extract_audio_from_video(file_path= filePath)
    #preprocessed_audio = preprocess_audio_series(raw_data= extracted_audio_raw)
    
    #Video
    # sampled = extract_N_video_frames(file_path= filePath, number_of_samples= 6)
    # resized_images = [resize_image(image= im, new_size= (248,140)) for im in sampled]
    # cropped_images = [crop_image_window(image= resi,training= training) / 255.0 for resi in resized_images]
    # preprocessed_video = np.stack(cropped_images)
    
    #Ground Truth
    video_gt = reading_label_data(file_name= file_name, dictionary= dictionary)
    #del extracted_audio_raw, sampled, resized_images, cropped_images
    #return (preprocessed_audio, preprocessed_video, video_gt)
    return video_gt

In [52]:
training_set_data = []
path = '/content/drive/MyDrive/Big5/train'
gt = pickle.load( open( "/content/drive/MyDrive/Big5/gt/annotation_training.pkl", "rb" ), encoding='latin1' )
t1 = datetime.datetime.utcnow()
#for filename in os.listdir(path):
filePath = '/content/drive/MyDrive/Big5/train/0ObEtA6Q8vU.005.mp4'
training_set_data.append(preprocessing_input(file_path= filePath, file_name= '0ObEtA6Q8vU.005.mp4', dictionary= gt, training= True))
t2 = datetime.datetime.utcnow()
#Measuring execution time
print('Elapsed time: ' + str(t2-t1))

Elapsed time: 0:00:00.000406


In [47]:
savename = 'training_set.dat'
with open(savename, "wb") as f:
    pickle.dump(training_set_data, f)

In [1]:
test_set_data = []
path = 'Big5/test'
gt = pickle.load( open( "Big5/gt/annotation_test.pkl", "rb" ), encoding='latin1' )
t1 = datetime.datetime.utcnow()
for filename in os.listdir(path):
    filePath = path+'/'+filename
    test_set_data.append(preprocessing_input(file_path= filePath, file_name= filename, dictionary= gt, training= False))
t2 = datetime.datetime.utcnow()
#Measuring execution time
print('Elapsed time: ' + str(t2-t1))

NameError: ignored

In [2]:
savename = 'test_set.dat'
with open(savename, "wb") as f:
    pickle.dump(test_set_data, f)

NameError: ignored

In [3]:
with open('training_set.dat', "rb") as training_file:
    train_set_data = pickle.load(training_file)

FileNotFoundError: ignored

In [4]:
with open('validation_set.dat', "rb") as validation_file:
    validation_set_data = pickle.load(validation_file)

FileNotFoundError: ignored

In [5]:
with open('test_set.dat', "rb") as test_file:
    test_set_data = pickle.load(test_file)

NameError: ignored

In [6]:
gt = pickle.load( open( "Big5/gt/annotation_training.pkl", "rb" ), encoding='latin1' )
neuroticism = list(gt['neuroticism'].values())
extraversion = list(gt['extraversion'].values())
agreeableness = list(gt['agreeableness'].values())
conscientiousness = list(gt['conscientiousness'].values())
openness = list(gt['openness'].values())

df = pd.DataFrame({'neuroticism': neuroticism,'extraversion': extraversion,'agreeableness':agreeableness,'conscientiousness':conscientiousness,'openness':openness})
df.plot(kind='density',xlim=(0,1))

NameError: ignored

In [7]:
print(len(train_set_data))
print(len(validation_set_data))
print(len(test_set_data))


NameError: ignored

In [8]:
train_random_index = random.randint(0, len(train_set_data)-1)
validation_random_index = random.randint(0, len(validation_set_data)-1)
test_random_index = random.randint(0, len(test_set_data)-1)

NameError: ignored

In [9]:
shape = train_set_data[train_random_index][0].shape
mfcc_train = train_set_data[train_random_index][0].reshape(shape[0],shape[1])

shape = validation_set_data[validation_random_index][0].shape
mfcc_validation = validation_set_data[validation_random_index][0].reshape(shape[0],shape[1])

shape = test_set_data[test_random_index][0].shape
mfcc_train = test_set_data[test_random_index][0].reshape(shape[0],shape[1])

mfccs = [mfcc_train,mfcc_validation,mfcc_train]

NameError: ignored

In [10]:
images_to_plot = []
images_to_plot.extend([(train_set_data[train_random_index][1])[i,:,:,:] for i in range(0,6)])
images_to_plot.extend([(validation_set_data[validation_random_index][1])[i,:,:,:] for i in range(0,6)])
images_to_plot.extend([(test_set_data[test_random_index][1])[i,:,:,:] for i in range(0,6)])
    

plt.figure(figsize=(18,11))

for i in range(18):
    plt.subplot(3,6,i+1)
    plt.imshow(images_to_plot[i])
plt.show()

NameError: ignored

In [11]:
librosa.display.specshow(mfccs[0], x_axis='time', y_axis='mel')

NameError: ignored